In [1]:
import numpy as np
import sys
import os
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings
from mp_functions import request_rates, request_ohlcv, compute_log_return
from mp_functions import request_cmc_historical
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

In [2]:
#load coin names - it removes those with some outliers in historical data
with open('/home/fbuonerba/codes/meta_data/new_coins.txt') as ff:
    coins=json.load(ff)
bad=['NPXS','MKR','VET','RHOC', 'ONT', 'ZIL', 'NANO', 'BAT','BCD','XTZ']
coins=np.array(coins)
where=[i for i in range(len(coins)) if coins[i] in bad]
coins=np.delete(coins, where)

prettybad=['BTC','BCN','DCR','BTG','BTM','XVG']
prettywhere=[i for i in range(len(coins)) if coins[i] in prettybad]
coins=np.delete(coins, prettywhere)
quotes=['BTC']
print(coins)

['ETH' 'XRP' 'BCH' 'EOS' 'XLM' 'LTC' 'ADA' 'XMR' 'IOTA' 'TRX' 'ETC' 'DASH'
 'NEO' 'XEM' 'BNB' 'ZEC' 'OMG' 'LSK' 'ZRX' 'QTUM' 'DOGE' 'BTS' 'DGB' 'ICX'
 'STEEM' 'AE' 'WAVES' 'SC' 'REP' 'PPT' 'GNT' 'STRAT']


In [ ]:
#####MAINTAINANCE OF EXCHANGE RATES#######
pathname='/home/fbuonerba/exchange_rates_data/'
exch=os.listdir(pathname)
EX={}
DEX={}
#create a dictionary coin ---> exchange rates coin/BTC listed in directory
#it looks only for those dated after 1515000000 - January 3rd 2018.
for coin in coins:
    EX[coin]=[x for x in exch if (coin+'_BTC' in x)]
    DEX[coin]=[]
    for x in EX[coin]:
        try:
            if float(x[-14:-4])>1515000000:
                DEX[coin].append(x)
        except:
            print('folder in wrong format - remove -----', x)
            os.unlink(pathname+x)

In [ ]:

#This window scans through the directory contiaing exchange rates and looks for
#wrong data. It then re-downloads data from coinAPI.
#REMARK: it seems BNB has real missing data.

#loop over dictionary values, to find wrong data (empty folder). Then download it.
for coin in coins:
    for exc in DEX[coin]:
        X=exc.split('.')
        X=X[0].split('_')
        X[-1]=int(X[-1])
        with open(pathname + exc) as infile:
            try:
                ex=json.load(infile)
                if ex=={} or ex=='{}':
                    print(exc, ex)
                    request_rates(X[-1], X[2], X[3])
            except Exception as e: 
                print(e, exc)
                request_rates(X[-1], X[2], X[3])
            

In [ ]:
########MAINTAINANCE OF LOG-RETURNS##########
pathname1='/home/fbuonerba/log_returns_data/'
log=os.listdir(pathname1)
LOG={}
DLOG={}
#create a dictionary coin --> list of log-returns folders for pair coin/BTC.
#it looks only for those dated after 1515000000 - January 3rd 2018.
for coin in coins:
    LOG[coin]=[x for x in log if (coin+'_BTC' in x)]
    DLOG[coin]=[]
    for x in LOG[coin]:
        try:
            if float(x[-14:-4])>1515000000:
                DLOG[coin].append(x)
        except:
            print('folder in wrong format - remove -----', x)
            os.unlink(pathname1+x)

In [ ]:
#maintainance of daily log-returns. 
#This window scans through the directory contiaing log-returns and looks for
#corrupted data. It then re-computes daily log-returns.
#REMARK: there are also many hourly log-returns. This window does not deal with them.

#loop over dictionary values, to find wrong data,by default NaN values. Then re-compute.
for coin in coins:
    for log in DLOG[coin]:
        X=log.split('.')
        X=X[0].split('_')
        X[-1]=int(X[-1])
        X[-2]=int(X[-2])
        with open(pathname1 + log) as infile:
            lg=json.load(infile)
            if np.isnan(lg)==True and X[-1]-X[-2]==86400:#focus on daily returns.
                print(log, lg)
                compute_log_return(X[-1], coin, 'BTC', X[-1]-X[-2])
                
                

In [ ]:
#This window check that every daily return is actually present in directory.
t0=1515024000 #jan 4th.
TIME=int((time.time()-t0)/86400 - 1)
for coin in coins:
    for t in range(TIME):
        init=t0+t*86400
        end=t0+(t+1)*86400
        symbol='log_return_'+coin+'_BTC_'+str(init)+'_'+str(end)+'.txt'
        if symbol not in DLOG[coin]:
            print(symbol)
            compute_log_return(init, coin, 'BTC', 86400)
    

In [ ]:
########MAINTAINANCE OF OHLCV##########
pathname3='/home/fbuonerba/ohlcv_data/'
ohlcv=os.listdir(pathname3)

OH={}
DOH={}
#create a dictionary coin ---> all ohlcv coin/BTC (for every exchange) listed in directory
#it looks only for those dated after 1515000000 - January 3rd 2018.
for coin in coins:
    OH[coin]=[x for x in ohlcv if (coin+'_BTC' in x)]
    DOH[coin]=[]
    for x in OH[coin]:
        if float(x[-14:-4])>1515000000:
            DOH[coin].append(x)

In [ ]:
#This window scans through directory to find missing ohlcv data, and in case downloads it. 
#Almost always, exchange QRYPTOS has missing data.
#DASH,NEO,XEM have a quite a few missing data points.
for coin in coins:
    for x in DOH[coin]:
        with open(pathname3 + x) as infile:
            oh=json.load(infile)
        if oh==[] or oh=='[]':
            x=x.split('.')
            x=x[0].split('_')
            x[-1]=int(x[-1])
            x[-2]=int(x[-2])
            if x[1]!='QRYPTOS':
                print(x)
                #request_ohlcv(x[-2], x[1]+'_'+x[2]+'_'+x[3]+'_BTC',1)

In [22]:
#########MAINTAINANCE OF COIN SUPPLY###########
#historical daily cmc data is impossible to correct because it is taken live.
#We can nevertheless check and eventually manually adjust.
pathname4='/home/fbuonerba/cmc_data/'
cmc=os.listdir(pathname4)
DCMC={}
for coin in coins:
    DCMC[coin]=[x for x in cmc if (coin in x)]
    for x in DCMC[coin]:
        try:
            with open(pathname4 + x) as infile:
                cm=json.load(infile)
            if cm=='{}' or cm=={}:
                print(x, cm)
            elif np.isnan(cm['circulating_supply'])==True:
                print('circ', x, cm)
        except:
            print(x)
            #only supply matters - can be manually filled.


In [6]:
#weekly cmc can be re-downloaded. If there are empty folders, or some folders
#contains NaN values, this window re-downloads the data for that whole day.
WEEK=[x for x in cmc if 'historical' in x]
WEEK.sort()
for x in WEEK:
    test=0
    X=x.split('.')
    ttime=float(X[0].split('_')[-1])
    utctime = datetime.utcfromtimestamp(ttime).strftime('%Y%m%d')
    with open(pathname4 + x) as infile:
        try:
            hist=json.load(infile)
            for coin in coins:
                if coin=='IOTA':
                    coin='MIOTA'
                if np.isnan(hist[coin]['supply'])==True:
                    print(coin, x)
                    test=1
            if test==1:
                request_cmc_historical(int(utctime))
        except:
            print('missing day', x)
            request_cmc_historical(int(utctime))